<a href="https://www.kaggle.com/code/ayushs9020/multiple-folds-multiple-models?scriptVersionId=137663960" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF0000; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF0000">1 | Approach 📊</p>

In [1]:
import warnings
warnings.filterwarnings("ignore")

<div style="border-radius:10px; border:#FF0000 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
* Make Multiple Folds of the data
* Use those folds on to train Multiple Models
* Watch each model closely and decide which is one is most efficient

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF7F00; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF7F00">2 | Data Preprocessing 📄️</p>

In [2]:
import pandas as pd 
import numpy as np
import tqdm
import os

<div style="border-radius:10px; border:#FF7F00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
In the competition we are given only $200$ Samples, I think that count is very low to train a model. Thanks to **[]()** for providing almost $6,500$ more samples for free 

In [3]:
train = pd.concat([
    pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv").drop("id" , axis = 1) , 
    pd.read_csv("/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv") , 
    pd.read_csv("/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv")
] , axis = 0)

train.head()

,prompt,A,B,C,D,E,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


<div style="border-radius:10px; border:#FF7F00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We cannot send `str()` columns to our models and thus we will try to first `Encode` them with number. Here we will the concept of `Ordinal Encoding`

In [4]:
for number , value in tqdm.tqdm(enumerate(train["answer"].unique()) , total = len(train["answer"].unique())):
    train["answer"] = np.where(train["answer"] == value , number , train["answer"])
print(train.shape)
train.head()

100%|██████████| 5/5 [00:00<00:00, 483.13it/s]

(6700, 7)


,prompt,A,B,C,D,E,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,0
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,1
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,1
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,2
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,0


<div style="border-radius:10px; border:#FF7F00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will train this `CSV` File, to make it accessible easier further, and also remove anomolies

In [5]:
os.makedirs("/kaggle/working/Pseudo Dir/Sample Train/")

train.to_csv("/kaggle/working/Pseudo Dir/Sample Train/Train.csv")

train = pd.read_csv("/kaggle/working/Pseudo Dir/Sample Train/Train.csv")

<div style="border-radius:10px; border:#FF7F00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

So we have a total of $6,700$ Samples. Lets make different folds of this data and save them in a directory, for further usage. Here we are making a total of $5$ Bags, which can be changed by changing the value of `n_bags`

In [6]:
os.makedirs("/kaggle/working/Pseudo Dir/Folds")

n_bags = 5

for bag in tqdm.tqdm(range(n_bags) , total = n_bags):
    
    train.sample(n = train.shape[0] , 
                 replace = True).to_csv("/kaggle/working/Pseudo Dir/Folds/Fold_" + str(bag) + ".csv")

100%|██████████| 5/5 [00:01<00:00,  4.77it/s]


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FFFF00; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FFFF00">3 | Tokenization Techniques 🔱</p>

In [7]:
import transformers
from transformers import AutoTokenizer , AutoModel

<div style="border-radius:10px; border:#FFFF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

For defining the encoder we first need to define a list of all the possibe choices

In [8]:
choices = [[
    train[column][index]
    for column in ["A" , "B" , "C" , "D" , "E"]]
    for index in range(train.shape[0])]

choices[0]

['MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."',
 'MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.',
 'MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.',
 'MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.',
 'MOND is a theory that eliminates the observed missing baryonic mass in galaxy clusters by imposing a new mathematical formulation of gravity that does not require the existence of dark matter.']

<div style="border-radius:10px; border:#FFFF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
$Tokenization$ is an `important aspect` of the $NLP$ tasks, it consists of `seperating informative` words in a `corpus of text` into a `numerical set`, which can be `further sent` to `Embedders` to apply `different` `Matrix Multiplication` to them and `getting correct Embeddments`
    
We will be using different Embedding Techniques here. At the end (after defining the Embeddings), we will try to make a function that will do the work for us, when we just have to choose which embedding we want

<div style="border-radius:10px; border:#FFFF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.1$ $|$ $RoBERTa$

* $RoBERTa$ $Base$

In [9]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

<div style="border-radius:10px; border:#FFFF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will make a loop that will do the tokenization for us 

In [10]:
os.makedirs("/kaggle/working/Pseudo Dir/Fold Embeds")

for bags in range(n_bags):
    
    tokens = np.empty(shape = train.shape[0] , 
                      dtype = transformers.tokenization_utils_base.BatchEncoding)
    
    train = pd.read_csv("/kaggle/working/Pseudo Dir/Folds/Fold_" + str(bags) + ".csv")
    
    for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):
        
        token = tokenizer([
            train["prompt"][index] , 
            train["prompt"][index] , 
            train["prompt"][index] , 
            train["prompt"][index] , 
            train["prompt"][index]
        ] , choices[index] , return_tensors = "np" , padding = True)
        
        tokens[index] = token
        
    np.save("/kaggle/working/Pseudo Dir/Fold Embeds/Folds_" + str(bags) , tokens)

100%|██████████| 6700/6700 [00:07<00:00, 896.63it/s]


In [11]:
tokens[1]

{'input_ids': array([[    0, 12375,    21, 13667, 11176, 20626,   116,     2,     2,
        47515, 22187, 12859,     7,     5, 10795,     9,  1403,    12,
        42116,  1743,     6,   147,   414,  4756,    31, 40617,    23,
         4460,   498, 18293, 37015,     7,     5,  7091,   414,   551,
           31, 40617,     9,   143,   656,    50,   423,    86,     4,
          152, 37015,    16,  4776,    30,    10,  1402,    86,    12,
        30231, 20572,   611, 11599, 15594,  3023,     4,     2,     1,
            1,     1,     1,     1,     1],
       [    0, 12375,    21, 13667, 11176, 20626,   116,     2,     2,
        47515, 22187, 12859,     7,     5,   786,    12,  3623, 23794,
            9,  1403,    12, 42116,  1743,     6,   147,   414,  4756,
           31, 40617,    23,  4460,   498,    16,  1122,     7,     5,
         7091,   414,   551,    31, 40617,     9,   143,   656,    50,
          423,    86,     4,   152, 37015,    16,  4776,    30,    10,
         1402,    8

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FF00; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FF00">4 | DataLoader 📂</p>

In [12]:
import torch 
from torch.utils.data import Dataset , DataLoader

<div style="border-radius:10px; border:#00FF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Lets first start by making a simple class. This class will inherit the `Dataset` class by `Pytorch`

In [13]:
class load_train(Dataset):pass

Now we will load the Embeds and their corresponding targets. Our data is divided, the Embeds are in a different folder as `NPY` file, and the targets are a column in `CSV` file. To kind of divide the complexity, instead of heaping it up at one place. We will only take the actual straight DIRs to load the data

In [14]:
class load_train(Dataset):
    
    def __init__(self , x_dir , y_dir):
        
        self.x = np.load(x_dir , allow_pickle = True)
        self.y = pd.read_csv(y_dir)["answer"]

Now we will some Getters so that we can access the data in Batches

In [15]:
class load_train(Dataset):
    
    def __init__(self , x_dir , y_dir):
        
        self.x = np.load(x_dir , allow_pickle = True)
        self.y = pd.read_csv(y_dir)["answer"]
        
    def __len__(self): return self.x.shape[0]
    
    def __getitem__(self , index): 
        
        inp_ids = torch.tensor
        
        embeds = self.x[index]
        targets = self.y[index]
        
        return embeds , targets

For now letss assume we have our `x_dir , y_dir` as 

In [16]:
x_dir = "/kaggle/working/Pseudo Dir/Fold Embeds/Folds_0.npy"
y_dir = "/kaggle/working/Pseudo Dir/Folds/Fold_0.csv"

In [17]:
sample_train = load_train(x_dir , y_dir)

sample_train_d = DataLoader(sample_train)

for x , y in tqdm.tqdm(sample_train_d , total = len(sample_train_d)):pass

100%|██████████| 6700/6700 [00:00<00:00, 6744.62it/s]


In [18]:
x , y

({'input_ids': tensor([[[    0,  2264,    16,  5082,  5269,   118,  9252,   298,   684,    13,
              11,     5, 16859,  4000,   539,   116,     2,     2,   534, 29059,
           17095,    16,    10,   538,  8376,  6381, 12086,    13,    63,  3493,
            3407, 23525,     8,  1632,  4002,     4,     2,     1,     1,     1],
          [    0,  2264,    16,  5082,  5269,   118,  9252,   298,   684,    13,
              11,     5, 16859,  4000,   539,   116,     2,     2,   534, 29059,
           17095,    16,    10,  4566,  1082,   684,    13,    63,  8178, 31565,
           23150,    31,     5,  3864, 36941,  8927,     4,     2,     1,     1],
          [    0,  2264,    16,  5082,  5269,   118,  9252,   298,   684,    13,
              11,     5, 16859,  4000,   539,   116,     2,     2,   534, 29059,
           17095,    16,    10,  4221,    14,  4542,    25,     5, 12329,   558,
              13,     5,  7247,  5410,  1951, 16058,  7169, 12329,     4,     2],
          [

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#0000FF; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #0000FF">5 | Model Setup 🔨</p>

<div style="border-radius:10px; border:#0000FF solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will define our model, remember to use the same type for `tokenizer`/`modelling`. Different models have different `Embedding Size` associated, which if gone different can lead to `indes out of range in self` `Errors`

In [19]:
from transformers import RobertaForMultipleChoice

<div style="border-radius:10px; border:#0000FF solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
## $5.1$ $|$ $RoBERTa$
    
$RobertaForMultipleChoice$ is a `PyTorch model` that can be used to `solve multiple choice questions`. It is a `fine-tuned version` of the $RoBERTa$ language model, which is a `powerful language model` that has been `trained on a massive dataset` of `text and code`.
$RobertaForMultipleChoice$ has a `multiple choice classification head` on top, which means that it can be used to `predict` the `correct answer` to a `multiple choice question`. The model takes as input the `question` and `possible answers`, and then `outputs` a `probability distribution`.

In [20]:
model = RobertaForMultipleChoice.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream t

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FFC0CB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FFC0CB">6 | Training Arguments 💾</p>

<div style="border-radius:10px; border:#FFC0CB solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
We will be using the all rounder Adam Optimizer

In [21]:
optim = torch.optim.Adam(model.parameters())

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#008080; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #008080">7 | Training Loop 💽</p>

<div style="border-radius:10px; border:#008080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Lets now train models. I am not using `Kaggle GPU` for some reason, but have imported the results from Wandb 
    
```
wandb.watch(model)

for bags in range(n_bags):

    losses = []

    x_dir = "/kaggle/working/Pseudo Dir/Fold Embeds/Folds_" + str(bags) + ".npy"
    y_dir = "/kaggle/working/Pseudo Dir/Folds/Fold_" + str(bags) + ".csv"

    train = load_train(x_dir , y_dir)

    train_d = DataLoader(train , shuffle = True)

    for x , y in tqdm.tqdm(train_d , total = len(train_d)):

        x = x.to("cuda")
        y = y.to("cuda")

        with torch.no_grad():preds = model(**x , labels = y)

        losses.append(preds.loss)

        wandb.log({"loss" : preds.loss})

        torch.cuda.empty_cache()

        optim.step()
```

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FFFF; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FFFF">8 | Result Visualization 🏆</p>

In [22]:
from IPython.display import IFrame

<div style="border-radius:10px; border:#00FFFF solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
## $8.1$ $|$ $Roberta$
    
loss	▆▇▆▆▆█▆▇▆▆▆▆▆▅▁▆▅▅▅▆▇▇▅▆▅▆▅▃▆▆▆▅▆▇▅▇▅▆▅▆

loss	1.59402

In [23]:
IFrame("https://wandb.ai/ayushsinghal659/Kaggle%20LLM%20%7C%20Multiple%20Models%20Folds%20%7C%20Albert/reports/ROBERTA-KAGGLE-LLM--Vmlldzo0OTUwODc3" , 1000 , 300)

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FF7F; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FF7F">9 | TO DO LIST 📝</p>

<div style="border-radius:10px; border:#00FF7F solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
* $TO$ $DO$ $1$ $:$ $ADD$ $MORE$ $MODELS$
* $TO$ $DO$ $2$ $:$ $IMPORVE$ $SCORES$
* $TO$ $DO$ $3$ $:$ $DANCE$

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF00AF; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF00AF">10 | Ending 🏁</p>

<div style="border-radius:10px; border:#FF00AF solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

**THIS IS NOT THE FULL IMPLEMENTATION, IT STILL LACKS MANY FUNCTIONALITIES AND IS VULENRABLE TO MANY EDGE CASES, WE WILL IMPROVE THIS IN THE UPCOMING VERSIONS**

**PLEASE COMMENT DOWN IF I DID ANY MISTAKES, OR IF CAN MAKE THIS MORE CONNECTED TO THE GROUND, OR SUGGESTIONS. YOUR ASSISTS ARE HIGHLY APPRECIABLE**

**THATS IT FOR TODAY GUYS**

**HOPE YOU UNDERSTOOD AND LIKED MY WORK**

**DONT FORGET TO MAKE AN UPVOTE $:)$**
    
<img src = "https://i.imgflip.com/19aadg.jpg">
   
**PEACE OUT**